In [2]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('max_columns', None)

# Load data

- 데이터 출처: 증권사 API, N사 금융, 금투협, 유료 데이터 벤더
- Section2: 파일 읽는 법, EDA

In [3]:
# 코드를 돌릴 때 warning이 안나오게 하기
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 영상에서는 fin_statement_2005_2017.csv이지만(데이터 문제가 있는 파일), 
# 해당 데이터에서 문제를 발견하여, fin_statement_new.csv라는 데이터(2006 ~ )로 대체되었습니다
df = pd.read_csv("my_data/fin_statement_new.csv")
df.head()

,Code,Name,결산월,상장일,시가총액,유통시가총액,"P/E(Adj., FY End)","P/B(Adj., FY End)","P/S(Adj., FY End)","P/FCF1(Adj., FY End)",EV/EBITDA,EV/EBIT,"DPS(보통주, 현금+주식, 연간)",year,수정주가,ROA,비유동부채,자산총계
0,A000020,동화약품,12,19760324,113122453500.000,65520525067.000,11.230,0.550,0.560,4.390,3.480,4.390,600.000,2006,6460,3.810,11961630.000,195293494.000
1,A000040,KR모터스,12,19760525,144125742600.000,111855988832.000,97.070,1.440,0.820,-7.780,16.840,40.670,0.000,2006,1098,-21.980,28190608.000,81250312.000
2,A000050,경방,12,19560303,275600000000.000,180269960000.000,426.230,0.860,2.010,5.780,15.340,20.660,0.000,2006,10339,-1.800,17344611.000,515345506.000
3,A000070,삼양홀딩스,12,19681227,602432236800.000,281576827480.000,8.360,0.570,0.530,11.160,4.600,6.210,1050.000,2006,63563,5.490,60878366.000,973804019.000
4,A000100,유한양행,12,19621101,1382097577000.000,1018467704491.000,25.280,2.670,4.070,-27.990,15.720,17.160,1000.000,2006,24911,7.530,95212024.000,746417156.000


In [5]:
# "12개월전대비수익률(현금배당포함)" 컬럼은 미리 제거하여 파일을 업로드했습니다
df = df.drop(["상장일"], axis=1)

In [6]:
df = df.rename(columns={
    "DPS(보통주, 현금+주식, 연간)": "DPS",
    "P/E(Adj., FY End)": "PER",
    "P/B(Adj., FY End)": "PBR",
    "P/S(Adj., FY End)": "PSR",
})

In [7]:
# 새로 올린 데이터는 2005가 아닌 2006부터 데이터가 존재합니다.
df.groupby(['year'])['Name'].count()
df.groupby(['Name'])['year'].count()

year
2006    1479
2007    1538
2008    1606
2009    1636
2010    1623
2011    1634
2012    1653
2013    1643
2014    1655
2015    1692
2016    1771
2017    1838
Name: Name, dtype: int64

Name
3S        12
3SOFT      4
3노드디지탈     6
AD모터스      7
AJS        9
          ..
흥국         8
흥국에프엔비     2
흥아해운      12
희림        12
희훈디앤지      4
Name: year, Length: 2241, dtype: int64

In [8]:
# code or name의 중복 체킹 방법1
df.groupby(['year'])['Name'].nunique().equals(df.groupby(['year'])['Code'].nunique())

True

In [9]:
# code or name의 중복 체킹 방법2
df.groupby(['year', 'Name'])['Code'].nunique()

year  Name  
2006  3S        1
      3SOFT     1
      AD모터스     1
      AJS       1
      AK홀딩스     1
               ..
2017  흥구석유      1
      흥국        1
      흥국에프엔비    1
      흥아해운      1
      희림        1
Name: Code, Length: 19768, dtype: int64

In [10]:
df.groupby(['year', 'Name'])['Code'].nunique().nunique()

1

# yearly returns

In [11]:
df[df['Name'] == '동화약품']

,Code,Name,결산월,시가총액,유통시가총액,PER,PBR,PSR,"P/FCF1(Adj., FY End)",EV/EBITDA,EV/EBIT,DPS,year,수정주가,ROA,비유동부채,자산총계
0,A000020,동화약품,12,113122453500.000,65520525067.000,11.230,0.550,0.560,4.390,3.480,4.390,600.000,2006,6460,3.810,11961630.000,195293494.000
1479,A000020,동화약품,12,162002526000.000,96942311558.000,18.390,0.920,1.000,12.290,6.720,8.690,700.000,2007,10720,3.950,12398017.000,192629355.000
3017,A000020,동화약품,12,289370029200.000,176371032797.000,14.630,1.760,1.800,-73.530,8.050,9.000,750.000,2008,6180,4.130,10897247.000,197948382.000
4623,A000020,동화약품,12,234624348000.000,138311053146.000,7.920,1.100,1.200,-2.990,5.440,5.940,100.000,2009,7580,10.280,8360598.000,220422961.000
6259,A000020,동화약품,12,167309505300.000,99833581813.000,31.520,1.010,1.460,-53.540,8.250,18.490,100.000,2010,5290,10.720,25682734.000,311472628.000
7882,A000020,동화약품,12,122898468000.000,75422789812.000,27.230,0.690,0.690,27.780,6.650,24.750,100.000,2011,4080,2.170,7427668.000,308991731.000
9516,A000020,동화약품,12,125691615000.000,77576864778.000,6.450,0.500,0.490,3.120,3.240,5.240,80.000,2012,6040,1.720,27437308.000,324009899.000
11169,A000020,동화약품,12,171499225800.000,108061662177.000,128.660,0.740,0.760,39.780,7.910,19.800,70.000,2013,4390,5.330,21947412.000,339431477.000
12812,A000020,동화약품,12,166192246500.000,105399122730.000,121.180,0.540,0.560,37.970,7.560,279.960,80.000,2014,5570,0.400,14690149.000,324284672.000
14467,A000020,동화약품,12,248869397700.000,155966451539.000,31.510,0.680,0.730,26.460,7.130,26.160,80.000,2015,8140,0.320,15561801.000,316165541.000


- Section4: `pivot()`

In [12]:
yearly_price_df = df.pivot(index="year", columns="Name", values="수정주가")
yearly_price_df.head()

Name       3S    3SOFT   3노드디지탈    AD모터스      AJS  AJ네트웍스  AJ렌터카     AK홀딩스  \
year                                                                         
2006  830.000 6530.000      NaN 1997.000 1250.000     NaN    NaN 12960.000   
2007  686.000 5410.000      NaN 1600.000 2200.000     NaN    NaN 26015.000   
2008  524.000  930.000  799.000  244.000  835.000     NaN    NaN  8859.000   
2009 1163.000    5.000 1770.000  911.000 2820.000     NaN    NaN 10362.000   
2010 3237.000      NaN 1840.000  718.000 1740.000     NaN    NaN 22039.000   

Name   APS홀딩스  AP시스템  AP우주통신  AP위성  BGF      BHK        BYC   C&S자산관리  \
year                                                                    
2006 2940.000    NaN 490.000   NaN  NaN 2777.000 150000.000 32870.000   
2007 2607.000    NaN  15.000   NaN  NaN 1575.000 210000.000 81308.000   
2008 2566.000    NaN     NaN   NaN  NaN  130.000 135500.000 47874.000   
2009 3185.000    NaN     NaN   NaN  NaN    5.000 125000.000 29304.000   
2010 5692.000    NaN     NaN   NaN  NaN      NaN 161000.000 21328.000   

Name     C&우방    C&중공업        CJ    CJ CGV  CJ E&M    CJ대한통운    CJ씨푸드  \
year                                                                    
2006 3450.000 3210.000 56213.000 19411.000     NaN 70275.000 1480.000   
2007 5010.000 5100.000 77794.000 11701.000     NaN 75750.000 1300.000   
2008  280.000  735.000 37718.000 13787.000     NaN 66750.000  540.000   
2009  185.000  180.000 59123.000 19955.000     NaN 56700.000 1040.000   
2010      NaN      NaN 74400.000 25398.000     NaN 94200.000 2365.000   

Name    CJ엔터테인      CJ오쇼핑     CJ인터넷     CJ제일제당   CJ프레시웨이  CJ헬로        CL  \
year                                                                       
2006 22300.000 142489.000 24300.000        NaN  9830.000   NaN 43650.000   
2007       NaN  96966.000 16100.000        NaN  8400.000   NaN 21150.000   
2008       NaN  72724.000 12100.000 179000.000  4450.000   NaN  2925.000   
2009       NaN 143414.000 13800.000 205500.000 14000.000   NaN  1900.000   
2010       NaN 231000.000 16200.000 217000.000 11700.000   NaN    30.000   

Name    CMG제약       CS  CSA 코스믹     CS홀딩스       CT&T     CU전자       DB  \
year                                                                     
2006 5842.000 7240.000      NaN 27267.000 138368.000 1853.000 1023.000   
2007 5144.000 4510.000 3530.000 37071.000  61619.000 1548.000 1388.000   
2008 1008.000 1835.000 3109.000 24772.000   5953.000  669.000  646.000   
2009 1315.000 3430.000 2863.000 39450.000   4762.000 1033.000 1249.000   
2010  713.000 7050.000 2417.000 64200.000   9950.000  730.000 1809.000   

Name     DB라이텍     DB하이텍       DMS    DRB동일  DSR    DSR제강         E1  \
year                                                                   
2006  1062.000 20200.000  7590.000 1101.000  NaN 1220.000  41950.000   
2007 12461.000  9640.000 10350.000 1159.000  NaN 2610.000 130000.000   
2008 10364.000  3780.000  6000.000  653.000  NaN 2135.000  69900.000   
2009  8628.000  7070.000  7730.000 3574.000  NaN 4640.000  67800.000   
2010  7710.000 10800.000 10500.000 2921.000  NaN 3915.000  54700.000   

Name        EG      EMW      F&F    FnC코오롱      GBS  GH신소재       GKL  \
year                                                                   
2006 10506.000 2708.000 4810.000 17600.000 8363.000    NaN       NaN   
2007 12351.000 1545.000 4185.000 24750.000 9041.000    NaN       NaN   
2008 10300.000  749.000 3000.000 10550.000  555.000    NaN       NaN   
2009 16600.000 3484.000 3175.000 12950.000    5.000    NaN       NaN   
2010 38000.000 3003.000 5560.000       NaN      NaN    NaN 20650.000   

Name      GK파워   GMR 머티리얼즈  GRT        GS      GST       GS건설     GS글로벌  \
year                                                                      
2006 34721.000 1071842.000  NaN 30100.000 3017.000  76842.000  4496.000   
2007 11222.000  968710.000  NaN 58100.000 3260.000 144253.000  4159.000   
2008  3154.000  165748.000  NaN 26900.000 1020.000  53632.000  3363

- rtn 구하기
    - $p_{n+1}\over{p_{n}}$ - 1

In [13]:
# 1. year_price_df.pct_change() == year_price_df / year_price_df.shift() - 1
# 2. `shift(-1)`을 하는 이유? 
#    - 데이터를 "xx년도에서 1년동안 들고있었더니, xx만큼 수익이 났다"로 해석하고 싶기 때문
yearly_rtn_df = yearly_price_df.pct_change(fill_method=None).shift(-1)
yearly_rtn_df.head()

Name     3S  3SOFT  3노드디지탈  AD모터스    AJS  AJ네트웍스  AJ렌터카  AK홀딩스  APS홀딩스  AP시스템  \
year                                                                            
2006 -0.173 -0.172     NaN -0.199  0.760     NaN    NaN  1.007  -0.113    NaN   
2007 -0.236 -0.828     NaN -0.848 -0.620     NaN    NaN -0.659  -0.016    NaN   
2008  1.219 -0.995   1.215  2.734  2.377     NaN    NaN  0.170   0.241    NaN   
2009  1.783    NaN   0.040 -0.212 -0.383     NaN    NaN  1.127   0.787    NaN   
2010  6.677    NaN  -0.504 -0.458 -0.353     NaN    NaN  0.154   0.702    NaN   

Name  AP우주통신  AP위성  BGF    BHK    BYC  C&S자산관리   C&우방  C&중공업     CJ  CJ CGV  \
year                                                                          
2006  -0.969   NaN  NaN -0.433  0.400    1.474  0.452  0.589  0.384  -0.397   
2007     NaN   NaN  NaN -0.917 -0.355   -0.411 -0.944 -0.856 -0.515   0.178   
2008     NaN   NaN  NaN -0.962 -0.077   -0.388 -0.339 -0.755  0.568   0.447   
2009     NaN   NaN  NaN    NaN  0.288   -0.272    NaN    NaN  0.258   0.273   
2010     NaN   NaN  NaN    NaN -0.043   -0.161    NaN    NaN -0.024  -0.048   

Name  CJ E&M  CJ대한통운  CJ씨푸드  CJ엔터테인  CJ오쇼핑  CJ인터넷  CJ제일제당  CJ프레시웨이  CJ헬로  \
year                                                                       
2006     NaN   0.078 -0.122     NaN -0.319 -0.337     NaN   -0.145   NaN   
2007     NaN  -0.119 -0.585     NaN -0.250 -0.248     NaN   -0.470   NaN   
2008     NaN  -0.151  0.926     NaN  0.972  0.140   0.148    2.146   NaN   
2009     NaN   0.661  1.274     NaN  0.611  0.174   0.056   -0.164   NaN   
2010     NaN  -0.204  0.142     NaN  0.117    NaN   0.334    1.137   NaN   

Name     CL  CMG제약     CS  CSA 코스믹  CS홀딩스   CT&T   CU전자     DB  DB라이텍  DB하이텍  \
year                                                                           
2006 -0.515 -0.119 -0.377      NaN  0.360 -0.555 -0.165  0.357 10.734 -0.523   
2007 -0.862 -0.804 -0.593   -0.119 -0.332 -0.903 -0.568 -0.535 -0.168 -0.608   
2008 -0.350  0.305  0.869   -0.079  0.593 -0.200  0.544  0.933 -0.168  0.870   
2009 -0.984 -0.458  1.055   -0.156  0.627  1.089 -0.293  0.448 -0.106  0.528   
2010    NaN -0.008  0.271   -0.026 -0.449 -0.962 -0.600 -0.522 -0.666 -0.329   

Name    DMS  DRB동일  DSR  DSR제강     E1     EG    EMW    F&F  FnC코오롱    GBS  \
year                                                                        
2006  0.364  0.053  NaN  1.139  2.099  0.176 -0.429 -0.130   0.406  0.081   
2007 -0.420 -0.437  NaN -0.182 -0.462 -0.166 -0.515 -0.283  -0.574 -0.939   
2008  0.288  4.473  NaN  1.173 -0.030  0.612  3.652  0.058   0.227 -0.991   
2009  0.358 -0.183  NaN -0.156 -0.193  1.289 -0.138  0.751     NaN    NaN   
2010 -0.475  0.510  NaN -0.152 -0.029  0.663 -0.481  0.205     NaN    NaN   

Name  GH신소재    GKL   GK파워  GMR 머티리얼즈  GRT     GS    GST   GS건설  GS글로벌  GS리테일  \
year                                                                           
2006    NaN    NaN -0.677     -0.096  NaN  0.930  0.081  0.877 -0.075    NaN   
2007    NaN    NaN -0.719     -0.829  NaN -0.537 -0.687 -0.628 -0.191    NaN   
2008    NaN    NaN -0.789     -0.706  NaN  0.257  0.701  0.871  2.867    NaN   
2009    NaN    NaN    NaN     -0.690  NaN  0.929  0.744  0.069 -0.051    NaN   
2010    NaN -0.119    NaN      0.351  NaN -0.222  0.180 -0.206 -0.404    NaN   

Name  GS홈쇼핑   GT&T  H1바이오  HB테크놀러지    HRS  IC코퍼레이션    IDH    IHQ   ISC  \
year                                                                     
2006 -0.190 -0.241 -0.064   -0.405  0.389   -0.678 -0.228 -0.507   NaN   
2007 -0.288 -0.589 -0.812   -0.788 -0.628   -0.674 -0.946 -0.541   NaN   
2008  0.793  0.553 -0.958    1.719  0.259   -0.810 -0.830 -0.318 0.579   
2009  0.274 -0.049    NaN    0.330  0.549      NaN    NaN  0.714 0.125   
2010  0.061 -0.295    NaN   -0.297 -0.337      NaN    NaN  1.119 0.690   

Name  ITX엠투엠     JS  JW생명과학   JW신약  JW중외제약  JW홀딩스  JYP Ent.  KB오토시스    KCC  \
year                                                                         
2006    

- 상장폐지 종목은 어떻게 처리가 되나?

In [14]:
yearly_price_df['AD모터스']

year
2006   1997.000
2007   1600.000
2008    244.000
2009    911.000
2010    718.000
2011    389.000
2012     69.000
2013        NaN
2014        NaN
2015        NaN
2016        NaN
2017        NaN
Name: AD모터스, dtype: float64

In [15]:
yearly_price_df['AD모터스'].pct_change(fill_method=None).shift(-1)

year
2006   -0.199
2007   -0.848
2008    2.734
2009   -0.212
2010   -0.458
2011   -0.823
2012      NaN
2013      NaN
2014      NaN
2015      NaN
2016      NaN
2017      NaN
Name: AD모터스, dtype: float64

- 2011/12에 매수했으면, 1년의 rtn value는은 보장됨.
- 2012/12에 매수했으면,
    - 2013년 1월에 상장폐지 되었을 수도 있고, 2013년 12월(초)에 되었을 수도 있기 때문에 => rtn이 nan처리됨

# Single Indicator(지표) backtesting

- Section1: `reset_index()`
- Section2: boolean selection, DataFrame arithmetic operation, dtype변환
- Section3: `groupby()` & `aggregation`
- Section4: `join()`, `pivot()`
- Section5: visualization

## DataFrame(matrix) Multiplication 복습

In [16]:
a = pd.DataFrame([[1,2], [3, np.nan,], [5,6]], columns=["a", "b"])
b = pd.DataFrame([[1,2], [3, 4,], [5,6]], columns=["a", "b"])*10
a
b

,a,b
0,1,2.000
1,3,NaN
2,5,6.000


,a,b
0,10,20
1,30,40
2,50,60


In [17]:
a * b

,a,b
0,10,40.000
1,90,NaN
2,250,360.000


In [18]:
a = pd.DataFrame([[1,2], [3, np.nan,], [5,6]], columns=["a", "b"])
b = pd.DataFrame([[1,2,3], [3, 4,5], [5,6,7]], columns=["c", "b", "d"])*10
a
b

,a,b
0,1,2.000
1,3,NaN
2,5,6.000


,c,b,d
0,10,20,30
1,30,40,50
2,50,60,70


In [19]:
a * b

,a,b,c,d
0,NaN,40.000,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,360.000,NaN,NaN


In [20]:
return_df = pd.DataFrame(
    [
        [np.nan,  np.nan, 2     ],
        [3,       np.nan, 3     ],
        [5,       6,      np.nan], 
    ],
    columns=["삼성", "현대", "SK"]
)
asset_on_df = pd.DataFrame(
    [
        [0, 1],
        [0, 1],
        [1, 0], 
    ],
    columns=["삼성", "SK"]
)
return_df
asset_on_df

,삼성,현대,SK
0,NaN,NaN,2.000
1,3.000,NaN,3.000
2,5.000,6.000,NaN


,삼성,SK
0,0,1
1,0,1
2,1,0


In [21]:
return_df * asset_on_df

,SK,삼성,현대
0,2.000,NaN,NaN
1,3.000,0.000,NaN
2,NaN,5.000,NaN


In [22]:
(return_df * asset_on_df).mean(axis=1)

0   2.000
1   1.500
2   5.000
dtype: float64

In [23]:
# 해결책
asset_on_df = asset_on_df.replace(0, np.nan)

In [24]:
return_df * asset_on_df

,SK,삼성,현대
0,2.000,NaN,NaN
1,3.000,NaN,NaN
2,NaN,5.000,NaN


In [25]:
# "동일가중" 방식의 투자인 경우, 포트폴리오 평균수익률 구하는 방법
(return_df * asset_on_df).mean(axis=1)

0   2.000
1   3.000
2   5.000
dtype: float64

## top_n

In [26]:
df.head()

,Code,Name,결산월,시가총액,유통시가총액,PER,PBR,PSR,"P/FCF1(Adj., FY End)",EV/EBITDA,EV/EBIT,DPS,year,수정주가,ROA,비유동부채,자산총계
0,A000020,동화약품,12,113122453500.000,65520525067.000,11.230,0.550,0.560,4.390,3.480,4.390,600.000,2006,6460,3.810,11961630.000,195293494.000
1,A000040,KR모터스,12,144125742600.000,111855988832.000,97.070,1.440,0.820,-7.780,16.840,40.670,0.000,2006,1098,-21.980,28190608.000,81250312.000
2,A000050,경방,12,275600000000.000,180269960000.000,426.230,0.860,2.010,5.780,15.340,20.660,0.000,2006,10339,-1.800,17344611.000,515345506.000
3,A000070,삼양홀딩스,12,602432236800.000,281576827480.000,8.360,0.570,0.530,11.160,4.600,6.210,1050.000,2006,63563,5.490,60878366.000,973804019.000
4,A000100,유한양행,12,1382097577000.000,1018467704491.000,25.280,2.670,4.070,-27.990,15.720,17.160,1000.000,2006,24911,7.530,95212024.000,746417156.000


In [27]:
indicator = "ROA"

In [28]:
top_n = 10

In [29]:
top_n_indicator_df = df.groupby(['year'])[indicator].nlargest(top_n).reset_index()
top_n_indicator_df.head()
top_n_indicator_df.tail()

,year,level_1,ROA
0,2006,1079,64.490
1,2006,1442,62.610
2,2006,1455,58.550
3,2006,1402,51.440
4,2006,1467,48.400


,year,level_1,ROA
115,2017,19658,52.050
116,2017,19696,48.980
117,2017,19730,46.440
118,2017,18265,41.830
119,2017,19729,39.480


In [30]:
# 종목 indexing
top_n_roa_df = df.loc[top_n_indicator_df['level_1']]
top_n_roa_df.head()

,Code,Name,결산월,시가총액,유통시가총액,PER,PBR,PSR,"P/FCF1(Adj., FY End)",EV/EBITDA,EV/EBIT,DPS,year,수정주가,ROA,비유동부채,자산총계
1079,A048870,시너지이노베이션,12,252240105600.000,186001853869.000,0.630,0.180,0.100,0.560,7.730,9.240,1050.000,2006,45925,64.490,484745.000,97283559.000
1442,A080000,에스엔유,12,177821980000.000,115242157510.000,11.850,3.280,3.440,-43.050,9.480,10.330,60.000,2006,4217,62.610,4739832.000,31210115.000
1455,A082800,루미마이크로,12,59698800000.000,28249472160.000,36.020,2.220,2.730,-15.020,14.720,41.420,0.000,2006,1311,58.550,3674557.000,19836533.000
1402,A075130,플랜티넷,12,127680000000.000,132685056000.000,27.240,5.140,10.070,51.340,21.080,26.370,110.000,2006,16200,51.440,1988671.000,26339925.000
1467,A084370,유진테크,12,31170176000.000,14724791142.000,NaN,0.000,NaN,NaN,NaN,NaN,65.000,2006,1447,48.400,1324699.000,14297125.000


In [31]:
indicator_df = top_n_roa_df.pivot(index="year", columns="Name", values="ROA")
indicator_df.head()

Name,DMS,GKL,ISC,KG이니시스,MP한강,NAVER,S&K폴리텍,SY,골프존뉴딘,나노캠텍,나노트로닉스,내츄럴엔도텍,네이쳐글로벌,넥스트리밍,넥스트바이오홀딩스,넥스트아이,넥스트칩,다믈멀티미디어,더블유게임즈,동양,동양네트웍스,디엔에프,디지텍시스템,라이온켐텍,루미마이크로,메지온,모다,모린스,미동앤씨네마,보해양조,부방,뷰웍스,비아트론,사파이어테크놀로지,삼성바이오로직스,서호전기,선데이토즈,성지건설,성창기업지주,셀트리온제약,슈프리마에이치큐,슈피겐코리아,시너지이노베이션,신세계,썸에이지,씨모텍,씨엘인터내셔널,씨젠,씨케이에이치,아나패스,아이디스홀딩스,아이디에스,아이즈비전,아트라스BX,앤디포스,에스디생명공학,에스엔유,엔스퍼트,영우디에스피,오성첨단소재,완리,우리넷,우리로,원익,웨이브일렉트로,위메이드,유니테스트,유진테크,이노와이어리스,이디디컴퍼니,이미지스,이아이디,인터불스,인터파크홀딩스,인포바인,인포피아,잇츠한불,제닉,제우스,제주반도체,조이맥스,종근당홀딩스,지디,지스마트글로벌,컴투스,코라오홀딩스,클리오,키위미디어그룹,키이스트,태산엘시디,테라리소스,테스,티엘아이,티이씨앤코,포시에스,플랜티넷,피엘에이,하이로닉,하이비젼시스템,한국가구,한국주강,한국타이어월드와이드,한솔로지스틱스,한화화인케미칼,현대시멘트,현대페인트,현대하이스코,화신테크,휴메딕스
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006,45.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.560,NaN,NaN,NaN,NaN,62.610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.400,46.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.020,41.640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,NaN,NaN,NaN,46.090,NaN,NaN,NaN,34.510,NaN,35.540,NaN,NaN,42.310,NaN,45.110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.900,NaN,NaN,NaN,NaN,NaN,NaN,41.620,NaN,NaN,NaN,NaN,NaN,92.180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,47.170,NaN,NaN,NaN,36.670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.530,39.340,NaN,NaN,NaN,45.310,48.830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.670,NaN,NaN,NaN,NaN,NaN,NaN,38.860,NaN,NaN,NaN,NaN,NaN,NaN,39.900,NaN,NaN,NaN,43.910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,NaN,NaN,NaN,NaN,NaN,41.180,NaN,NaN,NaN,NaN,38.390,NaN,NaN,NaN,45.910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.270,NaN,NaN,NaN,NaN,39.120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.080,NaN,NaN,35.860,NaN,NaN,NaN,NaN,NaN
2010,NaN,NaN,NaN,NaN,NaN,40.050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.520,NaN,NaN,NaN,42.220,NaN,NaN,NaN,NaN,NaN,63.530,NaN,NaN,45.940,NaN,NaN,NaN,NaN,NaN,NaN,41.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 주의: nan 값을 가지고 있는 종목은 아예 고려대상에서 배제됨(물론 agg 함수의 연산특성에 따라 다르기는하나, 대부분의 함수가 nan은 배제시키고 계산함)
- 깜짝 퀴즈
    - 각 row별, nan이 아닌 값이 정확히 top_n개 만큼 인지 확인하는 방법?


## backtest

In [32]:
indicator_df.head()

Name,DMS,GKL,ISC,KG이니시스,MP한강,NAVER,S&K폴리텍,SY,골프존뉴딘,나노캠텍,나노트로닉스,내츄럴엔도텍,네이쳐글로벌,넥스트리밍,넥스트바이오홀딩스,넥스트아이,넥스트칩,다믈멀티미디어,더블유게임즈,동양,동양네트웍스,디엔에프,디지텍시스템,라이온켐텍,루미마이크로,메지온,모다,모린스,미동앤씨네마,보해양조,부방,뷰웍스,비아트론,사파이어테크놀로지,삼성바이오로직스,서호전기,선데이토즈,성지건설,성창기업지주,셀트리온제약,슈프리마에이치큐,슈피겐코리아,시너지이노베이션,신세계,썸에이지,씨모텍,씨엘인터내셔널,씨젠,씨케이에이치,아나패스,아이디스홀딩스,아이디에스,아이즈비전,아트라스BX,앤디포스,에스디생명공학,에스엔유,엔스퍼트,영우디에스피,오성첨단소재,완리,우리넷,우리로,원익,웨이브일렉트로,위메이드,유니테스트,유진테크,이노와이어리스,이디디컴퍼니,이미지스,이아이디,인터불스,인터파크홀딩스,인포바인,인포피아,잇츠한불,제닉,제우스,제주반도체,조이맥스,종근당홀딩스,지디,지스마트글로벌,컴투스,코라오홀딩스,클리오,키위미디어그룹,키이스트,태산엘시디,테라리소스,테스,티엘아이,티이씨앤코,포시에스,플랜티넷,피엘에이,하이로닉,하이비젼시스템,한국가구,한국주강,한국타이어월드와이드,한솔로지스틱스,한화화인케미칼,현대시멘트,현대페인트,현대하이스코,화신테크,휴메딕스
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006,45.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.560,NaN,NaN,NaN,NaN,62.610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.400,46.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.020,41.640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,NaN,NaN,NaN,46.090,NaN,NaN,NaN,34.510,NaN,35.540,NaN,NaN,42.310,NaN,45.110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.900,NaN,NaN,NaN,NaN,NaN,NaN,41.620,NaN,NaN,NaN,NaN,NaN,92.180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,47.170,NaN,NaN,NaN,36.670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.530,39.340,NaN,NaN,NaN,45.310,48.830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.670,NaN,NaN,NaN,NaN,NaN,NaN,38.860,NaN,NaN,NaN,NaN,NaN,NaN,39.900,NaN,NaN,NaN,43.910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,NaN,NaN,NaN,NaN,NaN,41.180,NaN,NaN,NaN,NaN,38.390,NaN,NaN,NaN,45.910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.270,NaN,NaN,NaN,NaN,39.120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.080,NaN,NaN,35.860,NaN,NaN,NaN,NaN,NaN
2010,NaN,NaN,NaN,NaN,NaN,40.050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.520,NaN,NaN,NaN,42.220,NaN,NaN,NaN,NaN,NaN,63.530,NaN,NaN,45.940,NaN,NaN,NaN,NaN,NaN,NaN,41.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 포트폴리오 수익률 데이터

In [33]:
asset_on_df = indicator_df.notna().astype(int).replace(0, np.nan)
asset_on_df.head()

Name,DMS,GKL,ISC,KG이니시스,MP한강,NAVER,S&K폴리텍,SY,골프존뉴딘,나노캠텍,나노트로닉스,내츄럴엔도텍,네이쳐글로벌,넥스트리밍,넥스트바이오홀딩스,넥스트아이,넥스트칩,다믈멀티미디어,더블유게임즈,동양,동양네트웍스,디엔에프,디지텍시스템,라이온켐텍,루미마이크로,메지온,모다,모린스,미동앤씨네마,보해양조,부방,뷰웍스,비아트론,사파이어테크놀로지,삼성바이오로직스,서호전기,선데이토즈,성지건설,성창기업지주,셀트리온제약,슈프리마에이치큐,슈피겐코리아,시너지이노베이션,신세계,썸에이지,씨모텍,씨엘인터내셔널,씨젠,씨케이에이치,아나패스,아이디스홀딩스,아이디에스,아이즈비전,아트라스BX,앤디포스,에스디생명공학,에스엔유,엔스퍼트,영우디에스피,오성첨단소재,완리,우리넷,우리로,원익,웨이브일렉트로,위메이드,유니테스트,유진테크,이노와이어리스,이디디컴퍼니,이미지스,이아이디,인터불스,인터파크홀딩스,인포바인,인포피아,잇츠한불,제닉,제우스,제주반도체,조이맥스,종근당홀딩스,지디,지스마트글로벌,컴투스,코라오홀딩스,클리오,키위미디어그룹,키이스트,태산엘시디,테라리소스,테스,티엘아이,티이씨앤코,포시에스,플랜티넷,피엘에이,하이로닉,하이비젼시스템,한국가구,한국주강,한국타이어월드와이드,한솔로지스틱스,한화화인케미칼,현대시멘트,현대페인트,현대하이스코,화신테크,휴메딕스
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2006,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,NaN,NaN,NaN,1.000,NaN,NaN,NaN,1.000,NaN,1.000,NaN,NaN,1.000,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,1.000,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,1.000,NaN,NaN,NaN,1.000,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN
2010,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# 지난 영상 퀴즈 정답1
yearly_rtn_df.shape
asset_on_df.shape

(12, 2241)

(12, 109)

In [35]:
# 지난 영상 퀴즈 정답2
asset_on_df.notna().sum(axis=1)

year
2006    10
2007    10
2008    10
2009    10
2010    10
2011    10
2012    10
2013    10
2014    10
2015    10
2016    10
2017    10
dtype: int64

In [36]:
selected_return_df = yearly_rtn_df * asset_on_df
selected_return_df.head()

Name  3S  3SOFT  3노드디지탈  AD모터스  AJS  AJ네트웍스  AJ렌터카  AK홀딩스  APS홀딩스  AP시스템  \
year                                                                       
2006 NaN    NaN     NaN    NaN  NaN     NaN    NaN    NaN     NaN    NaN   
2007 NaN    NaN     NaN    NaN  NaN     NaN    NaN    NaN     NaN    NaN   
2008 NaN    NaN     NaN    NaN  NaN     NaN    NaN    NaN     NaN    NaN   
2009 NaN    NaN     NaN    NaN  NaN     NaN    NaN    NaN     NaN    NaN   
2010 NaN    NaN     NaN    NaN  NaN     NaN    NaN    NaN     NaN    NaN   

Name  AP우주통신  AP위성  BGF  BHK  BYC  C&S자산관리  C&우방  C&중공업  CJ  CJ CGV  CJ E&M  \
year                                                                          
2006     NaN   NaN  NaN  NaN  NaN      NaN   NaN    NaN NaN     NaN     NaN   
2007     NaN   NaN  NaN  NaN  NaN      NaN   NaN    NaN NaN     NaN     NaN   
2008     NaN   NaN  NaN  NaN  NaN      NaN   NaN    NaN NaN     NaN     NaN   
2009     NaN   NaN  NaN  NaN  NaN      NaN   NaN    NaN NaN     NaN     NaN   
2010     NaN   NaN  NaN  NaN  NaN      NaN   NaN    NaN NaN     NaN     NaN   

Name  CJ대한통운  CJ씨푸드  CJ엔터테인  CJ오쇼핑  CJ인터넷  CJ제일제당  CJ프레시웨이  CJ헬로  CL  CMG제약  \
year                                                                          
2006     NaN    NaN     NaN    NaN    NaN     NaN      NaN   NaN NaN    NaN   
2007     NaN    NaN     NaN    NaN    NaN     NaN      NaN   NaN NaN    NaN   
2008     NaN    NaN     NaN    NaN    NaN     NaN      NaN   NaN NaN    NaN   
2009     NaN    NaN     NaN    NaN    NaN     NaN      NaN   NaN NaN    NaN   
2010     NaN    NaN     NaN    NaN    NaN     NaN      NaN   NaN NaN    NaN   

Name  CS  CSA 코스믹  CS홀딩스  CT&T  CU전자  DB  DB라이텍  DB하이텍   DMS  DRB동일  DSR  \
year                                                                       
2006 NaN      NaN    NaN   NaN   NaN NaN    NaN    NaN 0.364    NaN  NaN   
2007 NaN      NaN    NaN   NaN   NaN NaN    NaN    NaN   NaN    NaN  NaN   
2008 NaN      NaN    NaN   NaN   NaN NaN    NaN    NaN   NaN    NaN  NaN   
2009 NaN      NaN    NaN   NaN   NaN NaN    NaN    NaN   NaN    NaN  NaN   
2010 NaN      NaN    NaN   NaN   NaN NaN    NaN    NaN   NaN    NaN  NaN   

Name  DSR제강  E1  EG  EMW  F&F  FnC코오롱  GBS  GH신소재  GKL  GK파워  GMR 머티리얼즈  GRT  \
year                                                                           
2006    NaN NaN NaN  NaN  NaN     NaN  NaN    NaN  NaN   NaN        NaN  NaN   
2007    NaN NaN NaN  NaN  NaN     NaN  NaN    NaN  NaN   NaN        NaN  NaN   
2008    NaN NaN NaN  NaN  NaN     NaN  NaN    NaN  NaN   NaN        NaN  NaN   
2009    NaN NaN NaN  NaN  NaN     NaN  NaN    NaN  NaN   NaN        NaN  NaN   
2010    NaN NaN NaN  NaN  NaN     NaN  NaN    NaN  NaN   NaN        NaN  NaN   

Name  GS  GST  GS건설  GS글로벌  GS리테일  GS홈쇼핑  GT&T  H1바이오  HB테크놀러지  HRS  IC코퍼레이션  \
year                                                                           
2006 NaN  NaN   NaN    NaN    NaN    NaN   NaN    NaN      NaN  NaN      NaN   
2007 NaN  NaN   NaN    NaN    NaN    NaN   NaN    NaN      NaN  NaN      NaN   
2008 NaN  NaN   NaN    NaN    NaN    NaN   NaN    NaN      NaN  NaN      NaN   
2009 NaN  NaN   NaN    NaN    NaN    NaN   NaN    NaN      NaN  NaN      NaN   
2010 NaN  NaN   NaN    NaN    NaN    NaN   NaN    NaN      NaN  NaN      NaN   

Name  IDH  IHQ   ISC  ITX엠투엠  JS  JW생명과학  JW신약  JW중외제약  JW홀딩스  JYP Ent.  \
year                                                                      
2006  NaN  NaN   NaN     NaN NaN     NaN   NaN     NaN    NaN       NaN   
2007  NaN  NaN   NaN     NaN NaN     NaN   NaN     NaN    NaN       NaN   
2008  NaN  NaN 0.579     NaN NaN     NaN   NaN     NaN    NaN       NaN   
2009  NaN  NaN   NaN     NaN NaN     NaN   NaN     NaN    NaN       NaN   
2010  NaN  NaN   NaN     NaN NaN     NaN   NaN     NaN    NaN       NaN   

Name  KB오토시스  KCC  KCC건설  KCI  KCTC  KCW  KC그린홀딩스  KC코트렐  KD건설  KEC  KG ETS  \
year                                                                          
2006     NaN  NaN   

In [37]:
selected_return_df.notna().sum(axis=1)

year
2006    10
2007    10
2008    10
2009    10
2010    10
2011    10
2012    10
2013    10
2014    10
2015     9
2016     8
2017     0
dtype: int64

In [38]:
a = asset_on_df.iloc[0]
a[a.notna()]

Name
DMS        1.000
루미마이크로     1.000
시너지이노베이션   1.000
아이디에스      1.000
에스엔유       1.000
유진테크       1.000
이노와이어리스    1.000
제우스        1.000
제주반도체      1.000
플랜티넷       1.000
Name: 2006, dtype: float64

In [39]:
b = yearly_rtn_df.iloc[0]
b[a[a.notna()].index]

Name
DMS         0.364
루미마이크로      1.516
시너지이노베이션   -0.632
아이디에스       0.126
에스엔유       -0.226
유진테크        0.653
이노와이어리스     0.025
제우스        -0.346
제주반도체      -0.293
플랜티넷       -0.615
Name: 2006, dtype: float64

In [40]:
rtn_series = selected_return_df.mean(axis=1)
rtn_series.head()

year
2006    0.057
2007   -0.606
2008    0.613
2009    0.019
2010   -0.118
dtype: float64

In [41]:
# 새로 수정된 데이터(fin_statement_new.csv)에서는 데이터 2006부터 시작하므로, 2005를 0으로 설정한 점에 주의바랍니다.
rtn_series.loc[2005] = 0
rtn_series = rtn_series.sort_index()
rtn_series

year
2005    0.000
2006    0.057
2007   -0.606
2008    0.613
2009    0.019
2010   -0.118
2011    0.192
2012   -0.056
2013    0.069
2014    0.026
2015   -0.060
2016    0.200
2017      NaN
dtype: float64

### 포트폴리오 누적 수익률 데이터

In [42]:
cum_rtn_series = (rtn_series + 1).cumprod().dropna()
cum_rtn_series

year
2005   1.000
2006   1.057
2007   0.417
2008   0.672
2009   0.685
2010   0.604
2011   0.720
2012   0.680
2013   0.727
2014   0.747
2015   0.702
2016   0.842
dtype: float64

In [43]:
pd.Series([1,2,3,4,5]).cumsum()

0     1
1     3
2     6
3    10
4    15
dtype: int64

In [44]:
%matplotlib inline
import matplotlib.pyplot as plt
 
fig, axes = plt.subplots(nrows=2, figsize=(15, 6), sharex=True)

axes[0].plot(cum_rtn_series.index, cum_rtn_series, marker='o');
axes[0].set_title("Cum return(line)");

axes[1].bar(rtn_series.index, rtn_series);
axes[1].set_title("Yearly return(bar)");

ModuleNotFoundError: No module named 'matplotlib'

## 함수화

In [ ]:
def get_return_series(selected_return_df):
    rtn_series = selected_return_df.mean(axis=1)
    rtn_series.loc[2005] = 0     # 주의: 영상속의 데이터와는 달리, 새로 업로드 된 데이터는 2006부터 존재하므로
                                 # 2004가 아니라 2005를 0으로 설정한 점에 주의해주세요
    rtn_series = rtn_series.sort_index()

    cum_rtn_series = (rtn_series + 1).cumprod().dropna()
    return rtn_series, cum_rtn_series

In [ ]:
def plot_return(cum_rtn_series, rtn_series):
    fig, axes = plt.subplots(nrows=2, figsize=(15, 6), sharex=True)
    axes[0].plot(cum_rtn_series.index, cum_rtn_series, marker='o');
    axes[1].bar(rtn_series.index, rtn_series);
    axes[0].set_title("Cum return(line)");
    axes[1].set_title("Yearly return(bar)");

In [ ]:
rtn_series, cum_rtn_series = get_return_series(selected_return_df)

In [ ]:
plot_return(cum_rtn_series, rtn_series)

## quantile (e.g. 상위 n% 종목 선정)

In [ ]:
quantile_by_year_series = df.groupby(['year'])[indicator].quantile(0.9)
quantile_by_year_series

In [ ]:
quantilie_indicator_df = df.join(quantile_by_year_series, how="left", on="year", rsuffix="_quantile")
quantilie_indicator_df.head(2)

In [ ]:
quantilie_indicator_df = quantilie_indicator_df[
    quantilie_indicator_df[indicator] >= quantilie_indicator_df["{}_quantile".format(indicator)]
]
quantilie_indicator_df.head()

In [ ]:
quantilie_indicator_df.groupby('year')['Code'].count()

In [ ]:
indicator_df = quantilie_indicator_df.pivot(index='year', columns="Name", values=indicator)
asset_on_df = indicator_df.notna().astype(int).replace(0, np.nan)

In [ ]:
selected_return_df = yearly_rtn_df * asset_on_df
selected_return_df.head()

In [ ]:
rtn_series, cum_rtn_series = get_return_series(selected_return_df)
plot_return(cum_rtn_series, rtn_series)

# 강환국님의 "할수있다 퀀트투자" 구현해보기

![](http://image.kyobobook.co.kr/images/book/large/392/l9791195887392.jpg)

## quantile + top10 
  

- Filter + Selector 구조 
    - Filter
        - e.g. 부채비율 0.5이상
        - 최종 포트폴리오 종목 갯수 선정에 직접적으로 영향 X
    - Selector
        - 최종적으로 xx개의 종목이 선택의 기준이 되는 indicator
        - e.g. PBR이 0.2 이상인 회사 중에 가장 낮은 순으로 20~30개 매수

- zipline (https://github.com/quantopian/zipline)

## [Chapter 6] 투자전략22. 소형주 + 저PBR 전략(200p)

- Filter
    - 소형주(시가총액 하위 20%)

- Select
    - PBR 0.2 이상
    - PBR이 가장 낮은 주식순으로 20~30개 매수

In [ ]:
#
# Filter
#
market_cap_quantile_series = df.groupby("year")['시가총액'].quantile(.2)

filtered_df = df.join(market_cap_quantile_series, on="year", how="left", rsuffix="20%_quantile")
filtered_df = filtered_df[filtered_df['시가총액'] <= filtered_df['시가총액20%_quantile']]
filtered_df.head()

In [ ]:
#
# Selector
#
filtered_df = filtered_df[filtered_df['PBR'] >= 0.2]

smallest_pbr_series = filtered_df.groupby("year")['PBR'].nsmallest(15)
smallest_pbr_series

In [ ]:
selected_index = smallest_pbr_series.index.get_level_values(1)

selector_df = filtered_df.loc[selected_index].pivot(
    index='year', columns="Name", values="PBR"
)
selector_df.head()

In [ ]:
asset_on_df = selector_df.notna().astype(int).replace(0, np.nan)
selected_return_df = yearly_rtn_df * asset_on_df

In [ ]:
rtn_series, cum_rtn_series = get_return_series(selected_return_df)
plot_return(cum_rtn_series, rtn_series)

## [Chapter 5] 투자전략20. 그레이엄의 마지막선물 업그레이드(188p)

- Filter
    - ROA 5% 이상
    - 부채비율 50% 이하
- Select
    - (PBR 0.2 이상)
    - PBR 낮은기업 20~30개 매수

In [ ]:
#
# Filter
#

# ROA >= 0.05
filtered_df = df[df['ROA'] >= 0.05]

# 부채비율 <= 0.5
filtered_df['부채비율'] = filtered_df['비유동부채'] / filtered_df['자산총계']
filtered_df = filtered_df[filtered_df['부채비율'] <= 0.5]

In [ ]:
#
# Selector(위의 투자전략22 것 그대로)
#
filtered_df = filtered_df[filtered_df['PBR'] >= 0.2]

smallest_pbr_series = filtered_df.groupby("year")['PBR'].nsmallest(15)
selected_index = smallest_pbr_series.index.get_level_values(1)

selector_df = filtered_df.loc[selected_index].pivot(
    index='year', columns="Name", values="PBR"
)

In [ ]:
asset_on_df = selector_df.notna().astype(int).replace(0, np.nan)
selected_return_df = yearly_rtn_df * asset_on_df

In [ ]:
rtn_series, cum_rtn_series = get_return_series(selected_return_df)
plot_return(cum_rtn_series, rtn_series)

## [Chapter 8] 투자전략24. 슈퍼가치전략(246p)

- Filter
    - 시가총액 하위 20%
- Selector
    - PBR, PCR, PER, PSR 순위를 매김
    - 각 순위를 sum을 해서 통합순위를 구함
    - 통합순위가 가장 높은 종목 50개 매수

In [ ]:
#
# Filter
#
market_cap_quantile_series = df.groupby("year")['시가총액'].quantile(.2)
filtered_df = df.join(market_cap_quantile_series, on="year", how="left", rsuffix="20%_quantile")
filtered_df = filtered_df[filtered_df['시가총액'] <= filtered_df['시가총액20%_quantile']]

In [ ]:
pd.Series([100, 1, 1, 3]).rank(method="max")
pd.Series([100, 1, 1, 3]).rank(method="min")

In [ ]:
pbr_rank_series = filtered_df.groupby("year")['PBR'].rank(method="max")
per_rank_series = filtered_df.groupby("year")['PER'].rank(method="max")
psr_rank_series = filtered_df.groupby("year")['PSR'].rank(method="max")

In [ ]:
psr_rank_series.head()

In [ ]:
psr_rank_series.sort_values().dropna().head()

In [ ]:
filtered_df = filtered_df.join(pbr_rank_series, how="left", rsuffix="_rank")
filtered_df = filtered_df.join(per_rank_series, how="left", rsuffix="_rank")
filtered_df = filtered_df.join(psr_rank_series, how="left", rsuffix="_rank")

In [ ]:
filtered_df['PBR_rank'].isna().sum()

- 어떻게 각 rank column의 nan을 메꿔야할까?

In [ ]:
filtered_df.filter(like="rank").columns

In [ ]:
#
# 주의: 종목을 선택하는 로직ㅇ[ 따라, '가장 작은 rank'로 부여하는게 타당할 수도 있고, '가장 큰 rank'로 부여하는 것이 타당할 수도 있습니다.
# 예를들어, PER이 작을수록 종목 선정에 우선 순위가 있도록 할 예정이고, PER이 작을수록 rank값이 작도록 설정했다면,
# PER이 nan인 종목들은 PER rank가 가장 큰 값(혹은 그 값보다 +1인 값)으로 메꿔져야 penalty를 받을 수 있습니다.
#

# 1. 0으로 메꾸는 법
filtered_df.loc[:, filtered_df.filter(like="rank").columns] = filtered_df.filter(like="rank").fillna(0)

# 2. 각 rank별 max 값 (혹은 그것보다 1 큰 값)으로 메꾸는 법
# filtered_df['PBR_rank'] = filtered_df['PBR_rank'].fillna(filtered_df['PBR_rank'].max() + 1)
# filtered_df['PER_rank'] = filtered_df['PER_rank'].fillna(filtered_df['PER_rank'].max() + 1)
# filtered_df['PSR_rank'] = filtered_df['PSR_rank'].fillna(filtered_df['PSR_rank'].max() + 1)

In [ ]:
filtered_df['rank_sum'] = filtered_df.filter(like="_rank").sum(axis=1)

In [ ]:
#
# Selector
#
max_rank_series = filtered_df.groupby("year")['rank_sum'].nlargest(15)
selected_index = max_rank_series.index.get_level_values(1)

selector_df = filtered_df.loc[selected_index].pivot(
    index='year', columns="Name", values="rank_sum"
)

In [ ]:
asset_on_df = selector_df.notna().astype(int).replace(0, np.nan)
selected_return_df = yearly_rtn_df * asset_on_df

In [ ]:
rtn_series, cum_rtn_series = get_return_series(selected_return_df)
plot_return(cum_rtn_series, rtn_series)

# 재무제표 기반 실전 프로젝트의 한계

- **(중요)Look ahead bias & Survivalship bias**
    - 특정 년도에 상장이 폐지가 되었다면 -> 바로 이전 년도에서 종목선정에 고려가 안됨
    - 즉, 이미 상장 폐지 정보를 미래 시점에서 확인하고, 해당 년도의 수익률을 nan으로 미리 메꾸어 버림
- Data availability(time alignment) 
    - 각 투자지표의 값들이 공시 되는 시기
        - 년도별, 분기별
    - 정확한 상장폐지 날짜?
- Data acquisition
    - 고정된 과거데이터로만 테스트 하면 안됨 -> 계속 새로운 데이터에 대한 갱신 필요
    - 크롤링, 증권사 API, 유료 데이터 벤더 등
- Data의 무결성
    - 아무리 증권사 API나 유로 벤더를 통해서 받아온 데이터라도, 문제가 있는 경우가 많음
    - 예를 들어, 일봉 OHLC -> C가 H보다 더 큰 경우 / 배당락, 주식분할 등의 이벤트가 제대로 반영이 안된 경우 등
    - 데이터의 결함, nunique==1, 비이상적인 값 등에 대한 EDA 필요
- 데이터가 년도별로만 존재하기 때문에, 1년에 한번 수익률이 찍혀서 변동성, MDD를 제대로 파악하기 어려움
- 거래세, 수수료 반영 X
    - 정확한 asset turnover 고려가 안됨
- 기타 위 실전예제에서의 한계
    - 데이터의 cleaning, validation 필요
    - Missing value에 대한 전처리 필요
    - 주어진 데이터 존재하지 않는 지표(column)은 다른 지표로 대체한 점

# 혼자 진행해보면 좋을 것들 

- `transform()`, `apply()` 함수 등을 구글링해서 독학해보기
- OOP 방식으로 구현해보기(확장성 있는 코드화)

In [ ]:
filter_list = [
    ColumnIndicator("부채비율", 0.5, lower_than=False),
    ColumnIndicator("ROE", 0.5, lower_than=True),
]
selector = Selector("PBR", 20, lowest=True)

backtest = Backtest(filter_list, selector, yearly_rtn_df)
backtest.run()

- 거래비용 주기

In [ ]:
for_positive_df = (yearly_rtn_df > 0).astype(int) * 0.99
for_negative_df = (yearly_rtn_df < 0).astype(int) * 1.01

extra_fee_considered_weight_df = for_positive_df + for_negative_df
yearly_rtn_df = yearly_rtn_df * extra_fee_considered_weight_df

- weight
    - 지금까지는 종목 선택 후, 다 동일가중 투자(`mean()`만으로 평균수익률을 구할 수 있었음)
    - e.g. ROA의 비중만큼 넣기 
        - Top n개의 종목을 산출 후, 각각의 값을 1,0(nan)으로 변환하기 전에, `selector_df`를 sum(axis=1)로 나눠주기 등

- 분기별 데이터 & 리벨런싱